# 24-1. 프로젝트 : 커스텀 프로젝트 직접 만들기

실습 코드에서 수행해 본 내용을 토대로, 이번에는 한국어 데이터셋에 도전해보겠습니다.

앞서 본 GLUE benchmark의 한국어 버전 KLUE benchmark를 들어보신 적 있나요?

GLUE와 마찬가지로 한국어 자연어처리에 대한 이해도를 높이기 위해 만들어진 데이터셋 benchmark입니다. 총 8가지의 데이터셋이 있습니다. 다만 이번 시간에 진행할 프로젝트는 KLUE의 dataset을 활용하는 것이 아닌, model(klue/ber-base)를 활용하여 NSMC(Naver Sentiment Movie Corpus) task를 도전해보겠습니다.

모델과 데이터에 관한 정보는 링크를 참조해주세요.

- [KLUE/Bert-base](https://huggingface.co/klue/bert-base)
- [NSMC](https://github.com/e9t/nsmc)

준비가 되셨다면 아래와 같은 순서로 진행해주세요.

### 라이브러리 버전을 확인해 봅니다.
---
사용할 라이브러리 버전을 둘러봅시다.

In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
---
데이터셋은 깃허브에서 다운받거나, [Huggingface datasets](https://huggingface.co/datasets)에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [2]:
#데이터셋 로드 및 구조 확인
from datasets import load_dataset

nsmc_dataset = load_dataset('nsmc')
print(nsmc_dataset)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
#데이터셋의 train 에서 샘플 추출
train = nsmc_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [4]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




In [6]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
# 각 데이터셋 파티션(트레인, 테스트)을 pandas DataFrame으로 변환
train_df = pd.DataFrame(nsmc_dataset['train'])
train_df, val_df = train_test_split(train_df, test_size=0.1)
test_df = pd.DataFrame(nsmc_dataset['test'])

# pandas DataFrame을 python dict 형식으로 변환
train_dict = train_df.to_dict('list')
val_dict = val_df.to_dict('list')
test_dict = test_df.to_dict('list')

# Huggingface Dataset 객체 생성
hf_train_dataset = Dataset.from_dict(train_dict)
hf_val_dataset = Dataset.from_dict(val_dict)
hf_test_dataset = Dataset.from_dict(test_dict)

# 데이터셋 샘플 확인
for i in range(5):
    print("Review:", hf_train_dataset[i]['document'])
    print("Label:", hf_train_dataset[i]['label'])
    print('\n')

Review: 극복할 수 없는 싼 티
Label: 0


Review: 인간의 악마성이 뭔지도 모르면서 악마를 논한다
Label: 0


Review: 더럽게 재미없고 기분더럽게하는 더러운~~~코메디ㅋㅋㅋㅋㅋ 아 진짜 마지막엔 뿜었음 다리저는것도 웃기냐 눈에키찔렸을때도 웃기고 칼로 푹푹쑤실때도 웃겨 ㅋㅋㅋㅋ
Label: 0


Review: 꼭봐라 이해안되면 두번봐라
Label: 1


Review: 난 과연 정상인걸까, 구속과 자유 자유와 구속에 대해 다시 한번 생각을 하게 해 준, 처음 만나는 자유. girl, interrupted
Label: 1




## STEP 2. klue/bert-base model 및 tokenizer 불러오기
---

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# klue/bert-base모델 및 토크나이저 불러오기
huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

# 모델 구조 확인
print(huggingface_model)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기
---

In [8]:
def transform_nsmc(batch):
    texts = batch['document']
    return huggingface_tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=128,  # 최대 길이 설정
        return_token_type_ids=False,
        return_attention_mask=True
    )

# Huggingface Dataset의 map 함수를 사용하여 데이터셋에 전처리 적용
hf_train_dataset = hf_train_dataset.map(transform_nsmc, batched=True)
hf_val_dataset = hf_val_dataset.map(transform_nsmc, batched=True)
hf_test_dataset = hf_test_dataset.map(transform_nsmc, batched=True)

  0%|          | 0/135 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [16]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 3e-5,                         #learning_rate
    per_device_train_batch_size = 32,   # 각 device 당 batch size
    per_device_eval_batch_size = 32,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
from datasets import load_metric
metric = load_metric('accuracy')

# 평가 메트릭 계산 함수 정의
def compute_metrics(eval_pred):    
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
# Trainer 설정
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments를 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,   # 평가 메트릭 함수
)

# 모델 학습
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 135000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4219


Epoch,Training Loss,Validation Loss,Accuracy
1,0.246600,0.234987,0.906133


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoi

TrainOutput(global_step=4219, training_loss=0.2624005410139469, metrics={'train_runtime': 2820.4495, 'train_samples_per_second': 47.865, 'train_steps_per_second': 1.496, 'total_flos': 8879998118400000.0, 'train_loss': 0.2624005410139469, 'epoch': 1.0})

In [19]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 32


{'eval_loss': 0.23845496773719788,
 'eval_accuracy': 0.90186,
 'eval_runtime': 348.9313,
 'eval_samples_per_second': 143.295,
 'eval_steps_per_second': 4.479,
 'epoch': 1.0}

In [20]:
del huggingface_model

## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
- 데이터 전처리, ```TrainingArguments``` 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.
---

### 데이터 전처리

In [30]:
import re
from konlpy.tag import Okt
from datasets import Dataset
import pandas as pd

# 데이터셋 로드
nsmc_dataset = load_dataset('nsmc')

def clean_text(text):
    # 한글과 공백을 제외하고 모두 제거
    text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text)
    return text

# 불용어 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

# 형태소 분석기 초기화
okt = Okt()

# 전처리 함수
def preprocess_data(dataset):
    cleaned_data = []
    for sentence in dataset:
        # 텍스트 정제
        sentence = clean_text(sentence)
        # 토큰화 및 불용어 제거
        tokenized = okt.morphs(sentence, stem=True)
        tokenized = [word for word in tokenized if not word in stopwords]
        cleaned_data.append(" ".join(tokenized))
    return cleaned_data

# 데이터셋 전처리 적용
train_data = preprocess_data(nsmc_dataset['train']['document'])
test_data = preprocess_data(nsmc_dataset['test']['document'])

# 전처리된 데이터를 DataFrame으로 변환
train_df = pd.DataFrame({'document': train_data, 'label': nsmc_dataset['train']['label']})
test_df = pd.DataFrame({'document': test_data, 'label': nsmc_dataset['test']['label']})

# Huggingface Dataset 객체 생성
hf_train_dataset = Dataset.from_dict(train_df.to_dict('list'))
hf_test_dataset = Dataset.from_dict(test_df.to_dict('list'))

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
- 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.

    - [Data Collator](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator)

    - [Trainer.TrainingArguments](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator)의 ```group_by_length```

- STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.
---

### training_arguments 조정

In [26]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DataCollatorWithPadding

output_dir = os.getenv('HOME')+'/aiffel/transformers'

# EarlyStopping 추가
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

training_arguments = TrainingArguments(
    output_dir=output_dir,                             # output이 저장될 경로
    evaluation_strategy="epoch",                       # evaluation하는 빈도
    save_strategy="epoch",
    learning_rate=3e-5,                                # learning_rate
    per_device_train_batch_size=32,                    # 각 device 당 batch size
    per_device_eval_batch_size=32,                     # evaluation 시에 batch size
    num_train_epochs=3,                                # train 시킬 총 epochs 3으로 늘림
    weight_decay=0.01,                                 # weight decay
    load_best_model_at_end=True,                       # 가장 좋은 모델을 끝에서 불러오기
    metric_for_best_model='accuracy',                  # 모델 평가 기준
    group_by_length=True,                     # Bucketing 활성화
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, Trainer

# 모델 로드
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

# 데이터 콜레이터 정의
data_collator = DataCollatorWithPadding(tokenizer=huggingface_tokenizer, pad_to_multiple_of=8)

# Trainer 설정
trainer = Trainer(
    model=huggingface_model,                   # 학습시킬 model
    args=training_arguments,                   # TrainingArguments를 통해 설정한 arguments
    data_collator=data_collator,               # 데이터 콜레이터 사용
    train_dataset=hf_train_dataset,            # training dataset
    eval_dataset=hf_val_dataset,               # evaluation dataset
    compute_metrics=compute_metrics,           # 평가 메트릭 함수
    callbacks=[early_stopping_callback]        # 콜백 추가
)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [31]:
# 모델 학습
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 135000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12657


Epoch,Training Loss,Validation Loss,Accuracy
1,0.252400,0.240403,0.903133
2,0.168600,0.252436,0.910267
3,0.095100,0.329424,0.909867


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 15000
  Batch size = 32
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-4219
Configuration saved in /aiffel/aiffel/transformers/checkpoint-4219/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-4219/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 15000
  Batch size = 32
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-8438
Configuration saved in /aiffel/aiffel/transformers/checkpoint-8438/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-8438/pytorch_model.bin
The following columns in the evaluation set  don't have a corres

TrainOutput(global_step=12657, training_loss=0.18482456883896656, metrics={'train_runtime': 8366.543, 'train_samples_per_second': 48.407, 'train_steps_per_second': 1.513, 'total_flos': 2.66399943552e+16, 'train_loss': 0.18482456883896656, 'epoch': 3.0})

In [34]:
# 사용할 모델에 맞는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

def tokenize_function(examples):
    return tokenizer(examples["document"], padding="max_length", truncation=True)

tokenized_train_dataset = hf_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = hf_test_dataset.map(tokenize_function, batched=True)
trainer.evaluate(hf_test_dataset)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/klue/bert-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/1a36e69d48a0

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 32


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [35]:
del huggingface_model

## 루브릭

| 평가요소 | 상세기준 |
|----------|----------|
| 1. 모델과 데이터를 정상적으로 불러오고, 작동하는 것을 확인하였다. | klue/bert-base를 NSMC 데이터셋으로 fine-tuning 하여, 모델이 정상적으로 작동하는 것을 확인하였다. |
| 2. Preprocessing을 개선하고, fine-tuning을 통해 모델의 성능을 개선시켰다. | Validation accuracy를 90% 이상으로 개선하였다. |
| 3. 모델 학습에 Bucketing을 성공적으로 적용하고, 그 결과를 비교분석하였다. | Bucketing task을 수행하여 fine-tuning 시 연산 속도와 모델 성능 간의 trade-off 관계가 발생하는지 여부를 확인하고, 분석한 결과를 제시하였다.|

1. 네 정상적을 작동했습니다.
2. accuracy가 90.6퍼센트가 나왔습니다.
3. Bucketing을 사용하게 되면 불필요한 계산을 줄여주므로 학습 시간 자체가 감소할 것이다. 하지만 학습 방법을 바꾸는 것은 아니므로 성능에 크게 영향을 미치지는 않을 것이다. 성능과 학습 시간 사이의 명확한 Tradeoff 관계는 나타나지 않을 것이다.

## 회고  
나만의 시도: earlystopping을 추가하였다.
배운점: glue와 nsmc 사용 과정에서 언어 자체가 다르다 보니 다양한 수정이 필요했다. 특히 데이터셋의 구조에서 차이가 있어서 NSMC는 단일 문장과 레이블로 구성되어 있어서 데이터 로딩, 처리, 모델입력 형식 등에서 차이가 있었다.
아쉬운점: 학습 시간이 너무 오래 걸려서 학습 과정 중에서 오류가 날 경우 다시 돌려야 해서 당황스러웠다.
느낀점: 뭔가 학습 시간이 확 늘어났다. 인공지능 개발은 인내를 필요로 하는 작업이란 것을 느꼇다